In [4]:
# increase Juypter notebook width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))

# Python Descriptors

By [GreatBahram](https://virgool.io/@greatbahram)

* I usually write about python at https://virgool.io/@GreatBahram/

![Our Society Logo](static/descriptors.png)

### Things to bear in mind

* Please, feel free to **shout out** if you have any question.


* This is not a normal presenation, it's going to be **an interactive tutorial**.

# Python Descriptors

![Flask-Sqlalchemy](static/sqlalchemy.png)

So, **What is** descriptors, **How** can we define one of them?

* A mechanism to **generalize** a property.
* According to python documentation:
> a descriptor is an object attribute with “binding behavior”

What is necessary to define a property?
* `__get__(self, instance, owner)`


* `__set__(self, instance, value)`


* `__delete__(self, instance)`

### Example 1

In [22]:
class NonNegative:
    """ A descriptors that forbids negative values."""

In [16]:
class Movie:
    rating = NonNegative(0)
    runtime = NonNegative(0)
    budget = NonNegative(0)
    gross = NonNegative(0)
    
    def __init__(self, title, rating, runtime, budget, gross):
        self.title = title
        self.rating = rating
        self.runtime = runtime
        self.budget = budget
        self.gross = gross
        
    def profit(self):
        return self.gross - self.budget

In [17]:
m = Movie('Titanic', rating=5, runtime=100, budget=100, gross=50)

### Example 2

```python
class NonNegative:
    """
     A descriptor that forbids negative values.
    """
    STORAGE = {}
    def __init__(self, initial):
        self.intial = initial
        
    def __get__(self, instance, owner):
        # we get here when someone calls x.d, and d is a NonNegative instance
        # instance = x
        # owner = type(x)
        return self.STORAGE.get(instance, self.default)
    
    def __set__(self, instance, value):
        if value < 0:
            raise ValueError("Negative value not allowed: %s" % value)
        self.STORAGE[instance] = value
    
    def __delete__(self, instance):
        del self.STORAGE[instance]
```

### Example 3

```python
class NonNegative:
    """
     A descriptor that forbids negative values.
     Store the data into the instance itself.
    """
    def __init__(self, initial):
        self.initial = initial
        
    def __get__(self, instance, owner):
        getattr(instance, '_value', self.initial)
        # return instance.__dict__.get('_value', self.initial)
    
    def __set__(self, instance, value):
        if value < 0:
            raise ValueError(f"Negative value not allowed: {value}")
        setattr(instance, '_value', value)
        # instance.__dict__['_value'] = value
```

### Example 4

* This only works on python 3.6+

```python
class NonNegative:
    """
     A descriptor that forbids negative values.
    """
    def __init__(self, initial):
        self.initial = initial
        self.name = None
        
    def __get__(self, instance, owner):
        getattr(instance, '_value', self.initial)
    
    def __set__(self, instance, value):
        if value < 0:
            raise ValueError(f"Negative value not allowed: {value}")
        if not name:
            self.name = '_value'
        setattr(instance, self.name, value)
    
    def __set_name__(self, owner, name):
        self.name  name
```

### Example 5 - Improve the implementation

```python
from abc import ABC, abstractmethod


class Validator(ABC):
    def __init__(self, initial):
        self.initial = initial
        
    def __get__(self, instance, owner):
        getattr(instance, '_value', self.initial)
    
    def __set__(self, instance, value):
        self.validate(value)
        setattr(instance, '_value', value)
    
    @abstractmethod
    def validate(self, value):
        """Validate the give data."""
```

## Exercise - cached_property

In [115]:
class Circle:
    def __init__(self, radius):
        self.radius = radius
    
    @cached_property
    def diameter(self):
        return self.radius * 2

* **Step 1**: A cached property should compute its value **once** and update its data by **assignment**.

In [123]:
class cached_property:
    """
    Decorator that converts a method with a single self argument into a
    property cached on the instance.
    """